In [1]:
pip install causal-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

data = pd.read_csv('C:/Users/ACER/Documents/TCDF-master/TCDF-master/data/Finance/manyinputs_returns30007000_header.csv')
print(data.head())

         S0        S1        S2        S3        S4        S5        S6  \
0  0.129999  0.991586  0.490534  1.030064  0.444978  1.208354  0.827543   
1  0.117729 -0.303972 -1.082923 -0.695533 -0.071504 -0.692810  0.102175   
2 -0.990932 -1.090502 -0.730285 -0.278989  0.345468 -0.935047 -0.460561   
3  1.940631  2.009416  1.683663  1.134586  1.794349  1.398274 -0.106631   
4  0.788248  0.093302 -0.032045  0.589187  0.616736  0.390370  0.965328   

         S7        S8        S9  ...       S15       S16       S17       S18  \
0  1.338010  0.759437  0.541619  ... -0.089528  0.507956  0.988139  0.987901   
1 -0.669950 -0.587785 -0.168711  ... -0.532579 -0.594185 -0.396401 -0.735928   
2 -0.421315  0.100106 -0.831737  ... -0.572841 -0.885619 -0.508891 -0.243055   
3  1.671531  1.490054  2.175950  ...  1.880062  1.461646  1.817153  1.704725   
4  0.542184 -0.203809  1.290336  ... -0.238918  0.637504  0.212562  0.933164   

        S19       S20       S21       S22       S23       S24  
0  0

In [3]:
import numpy as np
from causallearn.search.Granger.Granger import Granger
from sklearn.model_selection import train_test_split

df = pd.DataFrame(data)
data_array = df.values  
train, test = train_test_split(data_array, test_size=0.2, random_state=42)

G = Granger()
coeff_train = G.granger_lasso(train)
coeff_test = G.granger_lasso(test)

In [4]:
true_graph = coeff_train
estimated_graph = coeff_test

In [5]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Function to compute SHD
def compute_shd(true_graph, estimated_graph):
    if true_graph.shape != estimated_graph.shape:
        raise ValueError("The shapes of true_graph and estimated_graph must be the same.")
        
    added_edges = np.sum((estimated_graph - true_graph) == 1)
    removed_edges = np.sum((estimated_graph - true_graph) == -1)
    return added_edges + removed_edges

# Function to compute SID
def compute_sid(true_graph, estimated_graph):
    if true_graph.shape != estimated_graph.shape:
        raise ValueError("The shapes of true_graph and estimated_graph must be the same.")
        
    true_edges = (true_graph != 0).astype(int)
    estimated_edges = (estimated_graph != 0).astype(int)
    return np.sum(np.abs(true_edges - estimated_edges))

# Function to compute FDR
def compute_fdr(true_graph, estimated_graph):
    if true_graph.shape != estimated_graph.shape:
        raise ValueError("The shapes of true_graph and estimated_graph must be the same.")
        
    tp = np.sum((true_graph == 1) & (estimated_graph == 1))  # True Positives
    fp = np.sum((true_graph == 0) & (estimated_graph == 1))  # False Positives
    return fp / (tp + fp) if (tp + fp) > 0 else 0

# Function to compute MCC
def compute_mcc(true_graph, estimated_graph):
    if true_graph.shape != estimated_graph.shape:
        raise ValueError("The shapes of true_graph and estimated_graph must be the same.")
        
    # Flatten the matrices for binary classification
    y_true = (true_graph != 0).flatten()
    y_pred = (estimated_graph != 0).flatten()
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) if (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn) > 0 else 0
    return mcc


shd = compute_shd(true_graph, estimated_graph)
sid = compute_sid(true_graph, estimated_graph)
fdr = compute_fdr(true_graph, estimated_graph)
mcc = compute_mcc(true_graph, estimated_graph)

print("SHD:", shd)
print("SID:", sid)
print("FDR:", fdr)
print("MCC:", mcc)


SHD: 0
SID: 36
FDR: 0
MCC: 0.04973158663903936


In [6]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

estimated_adj_square = estimated_graph[:, :12]
G = nx.from_numpy_array(estimated_adj_square, create_using=nx.DiGraph)

# Visualize the graph
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G)  # Position nodes
nx.draw(G, pos, with_labels=True, node_size=700, node_color='lightblue', arrows=True)
plt.title("Causal Graph from Estimated Adjacency Matrix")
plt.show()


NetworkXError: Adjacency matrix not square: nx,ny=(25, 12)